# Stochastic Simulation

*Winter Semester 2023/24*

15.12.2023

Prof. Sebastian Krumscheid<br>
Asstistant: Stjepan Salatovic

<h3 align="center">
Exercise sheet 06
</h3>

---

<h1 align="center">
Variance Reduction Techniques
</h1>

In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np

from ipywidgets import interact
from scipy.stats import norm, multivariate_normal
from scipy.integrate import nquad
from typing import Tuple
from tqdm.notebook import tqdm

In [2]:
plt.rc('axes', labelsize=14)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=12)    # fontsize of the tick labels
plt.rc('ytick', labelsize=12)    # fontsize of the tick labels
plt.rc('legend', fontsize=14)    # legend fontsize

## Exercise 1

Suppose that we want to compute $p=\mathbb{P}(\boldsymbol{X}\in A)$, where
$\boldsymbol{X}$ is a $d$-dimensional Gaussian random vector with mean
$\boldsymbol{\mu} = \boldsymbol{0}$ and covariance matrix $\Sigma$. If
the (Borel) set $A\subseteq\mathbb{R}^d$ contains the mean $\boldsymbol{\mu}$,
then the event $\boldsymbol{X}\in A$ is typically not rare, and the
use of importance sampling is generally unnecessary. If, on the other
hand, $\boldsymbol{\mu}\not\in A$ and $p$ is small, then one may wish
to consider the use of importance sampling. Let $\mathbb{P}^\ast$ denote the
optimal (yet impractical) sampling measure with density $g^\ast$, that
is
\begin{equation*}
  d\mathbb{P}^\ast = g^\ast\,d\boldsymbol{x} = \frac{1}{\mathbb{P}(\boldsymbol{X}\in A)} \mathbb{I}_A \phi_{\Sigma}\,d\boldsymbol{x}\;,
\end{equation*}
where $\phi_{\Sigma}$ is the density of the
$\mathcal{N}(\boldsymbol{0},\Sigma)$ distribution. Given the rapid
decay of $\phi_{\Sigma}(\boldsymbol{x})$ as
${\Vert\boldsymbol{x} \Vert}_2\to \infty$, most of the mass of
$\mathbb{P}^\ast$ is typically located at the maximizer $\boldsymbol{x}^\ast$
of $\phi_{\Sigma}$ over $A$ (which we assume to exist uniquely). This
suggests using an importance sampling distribution $\tilde{\mathbb{P}}$ with
density $g$ that concentrates most of its mass near
$\boldsymbol{x}^\ast$, that makes $g$ easily computable, and from
which realizations can efficiently be generated. 

1. One such importance sampling distribution is the Gaussian
  distribution centered in $\boldsymbol{x}^\ast$ with covariance
  matrix $\Sigma$. Describe the importance sampling algorithm to
  estimate $p$.

2. Implement your algorithm for $d=2$. Take  $A=\bigl\{\boldsymbol{x}=(x_1,x_2)\in\mathbb{R}^2\colon x_i\ge
  a,\,i=1,2\bigr\}$ and
  \begin{equation*}
    \Sigma = \begin{pmatrix} 4 & -1\\ -1 & 4 \end{pmatrix}\;.
  \end{equation*}
  Then carry out the following points for $a=1,3,10$:
   1. First, try to provide simulation estimates of $p= \mathbb{P}(\boldsymbol{X}\in A)$ and the associated $95\%$ confidence interval using the (naive) crude Monte Carlo method.
    2. Next, find the point $\boldsymbol{x}^\ast$ that maximizes the $\mathcal{N}(\boldsymbol{0},\Sigma)$ density over $A$ and repeat point A., with the crude Monte Carlo method replaced by importance sampling, where the importance distribution is $\mathcal{N}(\boldsymbol{x}^\ast,\Sigma)$.
    3. In point B., experiment with importance distributions of the form $\mathcal{N}(\boldsymbol{x}^\ast,\delta \Sigma)$ for different $\delta>0$.

**Hint:** You can use [`scipy.integrate.nquad`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.nquad.html) for approximating the "true" value of $p$.

In [5]:
def crude_Monte_Carlo(N: int, a: float) -> float:
    """
    Estimates the probability of both components of a random variable X being greater
    than or equal to `a`, using a crude Monte Carlo simulation.
    Returns estimates for mean as well as variance.
    """
    # TODO
    return

In [6]:
def importance_sampling(N: int, a: float, delta: float=1.0, fix_seed: bool=False) -> float:
    """
    Estimates the probability of both components of a random variable X being greater
    than or equal to `a`, using importance sampling with a Gaussian distribution.
    Returns estimates for mean as well as variance.
    """
    # TODO
    return

## Exercise 2

Consider the problem of pricing a Barrier option with maturity $T>0$
based on the stock price $S$, which is given as the solution to the
stochastic differential equation
\begin{equation*}
  dS = rS\,dt + \sigma S\,dW\;,\quad S(0) = S_0\;,
\end{equation*}
where $W$ denotes a standard one-dimensional Wiener process. One can
show that $S_t = S_0e^{X_t}$, where
$X_t = (r-\sigma^2/2)t + \sigma W_t$ with $W$ being a standard Wiener
process. It follows that $S_t$ has a log-normal distribution for any
$t>0$. For $m\in\mathbb{N}$, let $t_i = i\Delta t$ with
$\Delta t = T/m$ denote the discrete observation times of the stock
price $S$ (e.g. daily at market closure). The payoff of a call
option subject to a lower barrier is then given by
\begin{equation*}
  \Psi(S_{t_0},S_{t_1},\dots, S_{T}) = {(S_T-K)}_{+}\mathbb{I}_{\bigl\{B\le \min_{i=0,\ldots,m}(S_{t_i})\bigr\}}\;,
\end{equation*}
where $B<S_0$ denotes the Barrier and $K\le S_0$ the strike
price. Here, $z_{+} = (\lvert z\rvert + z)/2$ denotes the positive
part of $z$. Estimate the expected payoff
$\mathbb{E}\bigl(\Psi(S_{t_0},S_{t_1},\dots, S_{T})\bigr)$ with antithetic
variables, using the process parameters $m=1000$, $r = 0.5$,
$\sigma = 0.3$, $T=2$, $S_0=5$, and $K=10$. Specifically, investigate
the variance reduction effect for different barrier values $B$.

In [9]:
def price_barrier_option(
    N: int,
    B: float,
    m: int,
    r: float,
    sigma: float,
    T: float,
    S0: float,
    K: float,
    antithetic: bool=False,
    fix_seed: bool=False,
) -> float:
    """
    Price a barrier option using crude (or antithetic) Monte Carlo
    estimation with `N` or (`N/2`) samples.
    Returns the mean as well as the variance of the estimator.
    """
    # TODO
    return

## Exercise 3

Consider the discrete time random walk
$\{X_n\in\mathbb{Z}\colon X_0=0, n\in\mathbb{N}\}$ with transition
probabilities:
\begin{align*}
p_{i,i+1} &\equiv\mathbb{P}(X_{n+1}=i+1\vert X_{n}=i) = a,\\
p_{i,i-1} &\equiv\mathbb{P}(X_{n+1}=i-1\vert X_{n}=i) =1-a,\quad n\ge 0,\,i\in\mathbb{Z},\ a\in(0,1),
\end{align*}
and define the stopping time $\tau_K := \inf\{n\colon X_n = K\}$ for a
given constant $K\in \mathbb{N}$. We aim at estimating $\mathbb{P}(\tau_K<T)$, for some given $T\in \mathbb{N}$.

1. Set $K=4$, $a=1/3$, $T=10$. Compute a Monte Carlo estimate of  $\mathbb{P}(\tau_K<T)$.

In [15]:
def estimate_prob(N: int, a: float, K: int, T: int, antithetic: bool=False) -> Tuple[float, float]:
    """
    Estimates the probability of a random walk reaching `K` within a given
    time horizon `T` based on `N` Monte Carlo samples.
    Returns the mean as well as the variance of the estimator.
    """
    # TODO
    return

2. For the same values as in the previous point, estimate $\mathbb{P}(\tau_K<T)$  using the antithetic variate variance reduction technique and compare your results to those in point 1. 